# Trampolin Analyse

### Imports

In [620]:
import pandas as pd
import numpy as np

### Load Event Results 

In [621]:
ce_str = "CE"
oe_str = "OE"

header = [np.array(["Mean", "Std", "Mean", "Std", "Mean", "Std", "Mean", "Std", "Mean", "Std", "Mean", "Std"]),
          np.array(["D", "D", "E", "E", "T", "T", "H", "H", "Pen", "Pen", "Total", "Total"])]

df_mean = pd.DataFrame(columns=header)

df_overview = pd.read_csv("data.csv")
event_names = ('2022EM', '2021WM', '2020WM', '2020OS')

genders = []
events = []
phases = []
routines = []
drop_outs = []
entries = []
for idx, row in df_overview.iterrows():
    if "Rimini" in row["Output"]:
        index_first_part = event_names[0]
    elif "Baku" in row["Output"]:
        index_first_part = event_names[1]
    elif "Tokyo2019" in row["Output"]:
        index_first_part = event_names[2]
    elif "Tokyo_2020" in row["Output"]:
        index_first_part = event_names[3]
    filename = row["Output"] + ".csv" 
    df = pd.read_csv(filename)
    if "Women" in row["Output"]:
        gender = "women"
    elif "Men" in row["Output"]:
        gender = "men"
    else:
        gender = ''

    if row["Event Phase"] == "qual":
        df_ce = df[df["Routine"]=="1st"]
        entries.append(len(df_ce["Name"].unique()))
        entries.append(len(df_ce["Name"].unique())) # has to be ce for both ce and oe, since sometimes oe hasn't even been tried
        drop_outs.append(np.sum(df_ce["H"] < 7))
        df_ce.drop(['Rank', 'Routine', 'Name','End','Country', 'Qualified'], axis=1, inplace=True)
        df_oe = df[df["Routine"]=="2nd"]
        drop_outs.append(np.sum(df_oe["H"] < 7))
        df_oe.drop(['Rank', 'Routine', 'Name','End','Country', 'Qualified'], axis=1, inplace=True)
    else:
        entries.append(len(df["Name"].unique()))
        df.drop(['Rank', 'Name','Country'], axis=1, inplace=True)
        drop_outs.append(np.sum(df["H"] < 7))
        
    
    
    

    df_ce = df_ce[df_ce["H"] >= 7]
    df_oe = df_oe[df_oe["H"] >= 7]
    df = df[df["H"] >= 7]

 


    
    if row["Event Phase"] == "qual":
        index_str = [index_first_part + " " + ce_str, index_first_part + " " + oe_str]
        data01 = np.zeros(len(df_ce.mean())*2)
        data02 = np.zeros(len(df_oe.mean())*2)
        data01[::2] = df_ce.mean().values.tolist()
        data01[1::2] = df_ce.std().values.tolist()
        data02[::2] = df_oe.mean().values.tolist()
        data02[1::2] = df_oe.std().values.tolist()
        data = [data01, data02]
        # data = [df_ce.mean().append(df_ce.std()).values.tolist(), df_oe.mean().append(df_oe.std()).values.tolist()]
        # print(data)
        df_overview = pd.DataFrame(np.reshape(data, (2,12)), columns=header)
        genders.append(gender)
        routines.append(ce_str)
        routines.append(oe_str)
        phases.append(row["Event Phase"])
        phases.append(row["Event Phase"])
        events.append(index_first_part)
    else:
        index_str = [index_first_part + " " + oe_str, gender]
        data = np.zeros(len(df.mean())*2)
        data[::2] = df.mean().values.tolist()
        data[1::2] = df.std().values.tolist()
        # data = df.mean().append(df.std()).values.tolist()
        # data = np.reshape(data, (1,12))
        df_overview = pd.DataFrame([data], columns=header)
        routines.append(oe_str)
        phases.append(row["Event Phase"])

    genders.append(gender)
    # phases.append(row["Event Phase"])
    events.append(index_first_part)
    df_mean = pd.concat([df_mean, df_overview])
    
df_mean['Gender'] = genders
df_mean['Event'] = events
df_mean['Phase'] = phases
df_mean['Routine'] = routines
df_mean['Entries'] = entries
df_mean['Dropouts'] = drop_outs
df_mean['Dropout Quote'] = (df_mean['Dropouts'] / df_mean['Entries']) * 100
# df_mean.set_index['Gender']

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:82: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:83: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:84: FutureWarning: Dropping of nuisance columns in 

In [622]:
df_mean.set_index(['Event', 'Phase', 'Routine'], inplace=True)
# df_mean.reindex(['qual', 'semi', 'final'], level='Phase')
df_mean.reset_index(inplace=True)
df_test = df_mean.pivot(index=['Event', 'Phase', 'Routine'], columns='Gender')

In [ ]:
df_test.columns.names = ['Estimator', 'Rating', 'Gender']


In [ ]:
df_test.reset_index(inplace=True)
df_test = pd.melt(df_test, id_vars=['Event', 'Phase', 'Routine'])


In [ ]:
df_test

,Event,Phase,Routine,Estimator,Rating,Gender,value
0,2020OS,final,OE,Mean,D,men,17.700000
1,2020OS,qual,CE,Mean,D,men,5.580000
2,2020OS,qual,OE,Mean,D,men,16.870000
3,2020WM,final,OE,Mean,D,men,17.471429
4,2020WM,qual,CE,Mean,D,men,5.728723
...,...,...,...,...,...,...,...
355,2021WM,semi,OE,Std,Total,women,2.507889
356,2022EM,final,OE,Std,Total,women,1.094177
357,2022EM,qual,CE,Std,Total,women,3.127149
358,2022EM,qual,OE,Std,Total,women,2.608951


In [ ]:

newOrder = [0, 2, 1, 3, 4, 6, 5, 7, 8, 10, 9, 11, 12, 14, 13, 15, 16, 18, 17, 19, 20, 22, 21, 23]
test = df_test.pivot(index=['Event', 'Phase', 'Routine'], columns=['Rating', 'Gender', 'Estimator'], values= 'value')
test = test[test.columns[newOrder]]
test = test.reindex(['qual', 'semi', 'final'], level='Phase')



In [ ]:
test.to_csv("Table.csv")


In [ ]:
test

Rating                        D                                         E  \
Gender                      men                women                  men   
Estimator                  Mean       Std       Mean       Std       Mean   
Event  Phase Routine                                                        
2020OS qual  CE        5.580000  0.923657   5.007143  0.410374  18.013333   
             OE       16.870000  1.367926  13.900000  1.484587  15.530000   
       final OE       17.700000  0.374166  14.425000  0.761108  16.042857   
2020WM qual  CE        5.728723  0.607590   4.793333  0.679494  17.436170   
             OE       15.662500  1.609259  12.555072  1.943355  15.033750   
       semi  OE       17.059091  0.574588  13.994737  0.440295  15.554545   
       final OE       17.471429  0.402965  14.400000  0.305505  16.071429   
2021WM qual  CE        5.586111  0.425691   4.576923  0.772217  17.286111   
             OE       15.694915  1.501772  12.779070  1.394472  14.413559   
       semi  OE       16.882353  0.959320  13.678947  1.034719  15.188235   
       final OE       17.762500  0.597465  14.000000  1.135153  15.775000   
2022EM qual  CE       15.307547  1.193065  11.231111  1.696497  14.328302   
             OE       15.017073  1.648621  11.537209  1.415059  14.141463   
       semi  OE       16.168421  0.623656  12.347368  1.421020  14.831579   
       final OE       16.550000  0.362531  13.500000  0.911043  15.125000   

Rating                                                       T            ...  \
Gender                              women                  men            ...   
Estimator                  Std       Mean       Std       Mean       Std  ...   
Event  Phase Routine                                                      ...   
2020OS qual  CE       1.242616  17.514286  0.663159  17.720667  1.232345  ...   
             OE       1.040353  15.109091  1.214458  16.735000  0.818715  ...   
       final OE       0.692477  15.587500  0.833988  17.162857  0.385377  ...   
2020WM qual  CE       1.091891  16.694667  1.438458  17.350426  0.862019  ...   
             OE       1.301017  14.747826  1.096868  16.276063  0.871170  ...   
       semi  OE       0.581236  15.057895  1.058466  16.894091  0.498301  ...   
       final OE       0.819988  15.628571  0.368394  17.243571  0.524005  ...   
2021WM qual  CE       0.942323  17.057692  0.936691  17.290347  0.703622  ...   
             OE       1.148081  15.088372  1.098529  16.103305  0.874733  ...   
       semi  OE       1.163982  15.263158  1.061556  16.677941  0.895763  ...   
       final OE       0.667083  16.000000  0.684523  17.139375  0.264095  ...   
2022EM qual  CE       1.260094  14.973333  1.198180  16.126226  0.702578  ...   
             OE       1.177067  14.748837  1.011216  16.035610  1.112407  ...   
       semi  OE       0.732615  15.221053  0.734529  16.446316  0.471702  ...   
       final OE       1.031988  15.657143  0.531843  16.798750  0.236851  ...   

Rating                       H                 Pen                      \
Gender                   women                 men               women   
Estimator                 Mean       Std      Mean       Std      Mean   
Event  Phase Routine                                                     
2020OS qual  CE       9.364286  0.264886  0.000000  0.000000 -0.028571   
             OE       9.236364  0.475968 -0.040000  0.126491 -0.018182   
       final OE       9.087500  0.461171  0.000000  0.000000  0.000000   
2020WM qual  CE       9.402667  0.399991 -0.067021  0.354203 -0.008000   
             OE       9.153623  0.403507 -0.010000  0.043864 -0.026087   
       semi  OE       9.252632  0.327225  0.000000  0.000000 -0.010526   
       final OE       9.200000  0.336650  0.000000  0.000000  0.000000   
2021WM qual  CE       9.336538  0.285622  0.000000  0.000000 -0.042308   
             OE       9.227907  0.334753 -0.006780  0.036504 -0.004651   
       semi  OE       9.100000  0.

In [632]:
df_dropout = df_mean.copy()
df_dropout.columns = df_mean.columns.droplevel(0)
df_dropout


,,,,D,D,E,E,T,T,H,H,Pen,Pen,Total,Total,,,,
0,2022EM,qual,CE,11.231111,1.696497,14.973333,1.198180,14.496444,0.852953,9.162222,0.509585,0.000000,0.000000,49.863111,3.127149,women,52,7,13.461538
1,2022EM,qual,OE,11.537209,1.415059,14.748837,1.011216,14.440233,0.755842,9.165116,0.455592,-0.018605,0.095757,49.872791,2.608951,women,52,13,25.000000
2,2022EM,semi,OE,12.347368,1.421020,15.221053,0.734529,14.736842,0.477133,9.173684,0.260004,0.000000,0.000000,51.478947,2.205589,women,24,5,20.833333
3,2022EM,final,OE,13.500000,0.911043,15.657143,0.531843,15.192857,0.342380,9.142857,0.207020,0.000000,0.000000,53.492857,1.094177,women,8,1,12.500000
4,2022EM,qual,CE,15.307547,1.193065,14.328302,1.260094,16.126226,0.702578,9.120755,0.377420,-0.011321,0.046659,54.871509,2.713526,men,59,6,10.169492
5,2022EM,qual,OE,15.017073,1.648621,14.141463,1.177067,16.035610,1.112407,9.007317,0.622652,-0.004878,0.031235,54.196585,4.008539,men,59,21,35.593220
6,2022EM,semi,OE,16.168421,0.623656,14.831579,0.732615,16.446316,0.471702,9.194737,0.267652,-0.052632,0.130675,56.588421,1.446245,men,24,5,20.833333
7,2022EM,final,OE,16.550000,0.362531,15.125000,1.031988,16.798750,0.236851,9.075000,0.190863,0.000000,0.000000,57.548750,1.327387,men,8,0,0.000000
8,2021WM,qual,CE,5.586111,0.425691,17.286111,0.942323,17.290347,0.703622,9.391667,0.266300,0.000000,0.000000,49.554236,1.733623,men,78,6,7.692308
9,2021WM,qual,OE,15.694915,1.501772,14.413559,1.148081,16.103305,0.874733,9.181356,0.483334,-0.006780,0.036504,55.386356,3.435637,men,78,19,24.358974


In [633]:
df_dropout = df_dropout.drop(['D', 'E', 'T', 'H', 'Pen', 'Total'], axis=1)
df_dropout


,,,,,,,
0,2022EM,qual,CE,women,52,7,13.461538
1,2022EM,qual,OE,women,52,13,25.000000
2,2022EM,semi,OE,women,24,5,20.833333
3,2022EM,final,OE,women,8,1,12.500000
4,2022EM,qual,CE,men,59,6,10.169492
5,2022EM,qual,OE,men,59,21,35.593220
6,2022EM,semi,OE,men,24,5,20.833333
7,2022EM,final,OE,men,8,0,0.000000
8,2021WM,qual,CE,men,78,6,7.692308
9,2021WM,qual,OE,men,78,19,24.358974


In [634]:
df_dropout.columns = ['Event', 'Phase', 'Routine', 'Gender', 'Entries', 'Dropouts', 'Dropout Rate']


In [635]:
# df_dropout.reset_index(inplace=True)
df_test2 = df_dropout.pivot(index=['Event', 'Phase', 'Routine'], columns='Gender')

In [637]:
df_test2 = df_test2.reindex(['qual', 'semi', 'final'], level='Phase')

In [639]:
df_test2
df_test2.to_csv('Dropout.csv')